In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import StreamingStdOutCallbackHandler

### Settings
chat = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


# embeddings 저장 위치
cache_dir = LocalFileStore("./.cache")

# file을 seperator와 token 사이즈를 기준으로 split
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

file = UnstructuredFileLoader("./files/chapter_three.txt")

docs = file.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vector_store = FAISS.from_documents(docs, cached_embeddings)

retriver = vector_store.as_retriever()

### Chain
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a helpful assistant. Answer quetions using only the"
                " following context. If you don't know the answer just say you"
                " don't know, don't make it up:\n\n{context}"
            ),
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | chat
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [2]:
invoke_chain("Is Aaronson guilty?")

Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [5]:
invoke_chain("Have I ever asked following question? Is Aaronson guilty?")

Yes, you have asked if Aaronson is guilty.Yes, you have asked if Aaronson is guilty.


In [3]:
invoke_chain("What message did he write in the table?")

He wrote "2+2=5" on the table.He wrote "2+2=5" on the table.


In [4]:
invoke_chain("What message did he write in the table?")

He wrote "2+2=5" on the table.He wrote "2+2=5" on the table.
